In [2]:
!pip install pandas
!pip install py2neo


In [21]:
import pandas as pd
from py2neo import Graph, Node, Relationship

In [22]:
node_df = pd.read_csv('hetionet/nodes.tsv', sep='\t')
edges_df = pd.read_csv('hetionet/edges.tsv', sep='\t')

Make connection to Neo4j database

In [23]:
graph = Graph("bolt://localhost:7687", auth=('neo4j', 'lucky238'))

Import node data into graph database

In [14]:
def create_Node(row):
    node_label, identifier = row['id'].split('::')

    if 'Anatomy' == node_label:
        anatomy_node = Node(label = node_label, id = identifier, name = row['name'])
        graph.merge(anatomy_node, 'Anatomy', 'id')
    
    elif 'Disease' == node_label:
        disease_node = Node(label = node_label, id= identifier, name = row['name'])
        graph.merge(disease_node, 'Disease', 'id')

    elif 'Compound' == node_label:
        compound_node = Node(label = node_label, id= identifier, name = row['name'])
        graph.merge(compound_node, 'Compound', 'id')
    
    elif 'Gene' == node_label:
        gene_node = Node(label = node_label, id = identifier, name = row['name'])
        graph.merge(gene_node, 'Gene', 'id')

In [19]:
node_df.apply(create_Node, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
23031    None
23032    None
23033    None
23034    None
23035    None
Length: 23036, dtype: object

Import relationship data into graph database

In [16]:
def get_interaction_description(abbreviation):
    interaction_map = {
        'GiG': 'interacts_GiG',
        'CrC': 'resembles_CrC',
        'DdG': 'downregulates_DdG',
        'DlA': 'localizes_DlA',
        'CtD': 'treats_CtD',
        'CbG': 'binds_CbG',
        'CuG': 'upregulates_CuG',
        'DrD': 'resembles_DrD',
        'DaG': 'associates_DaG',
        'CpD': 'palliates_CpD',
        'AdG': 'downregulates_AdG',
        'AuG': 'upregulates_AuG',
        'GcG': 'covaries_GcG',
        'Gr>G': 'regulates_Gr>G',
        'CdG': 'downregulates_CdG',
        'DuG': 'upregulates_DuG',
        'AeG': 'expresses_AeG',
    }
    return interaction_map.get(abbreviation)

In [17]:
def create_Relationship(row):
    source_type, source_id = row['source'].split('::')
    target_type, target_id = row['target'].split('::')
    rel_type = get_interaction_description(row['metaedge'])


    source_node = graph.nodes.match(source_type, id=source_id).first()
    target_node = graph.nodes.match(target_type, id=target_id).first()

    if source_node and target_node:
        rel = Relationship(source_node, rel_type, target_node)
        graph.merge(rel)
    else:
        print("Relationship not found")

In [20]:
edges_df.apply(create_Relationship, axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
1292198    None
1292199    None
1292200    None
1292201    None
1292202    None
Length: 1292203, dtype: object